# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 45, 1739, 15, 18, 2, 443, 2, 696, 1850, 2108, 177, 2911, 573, 85, 74, 323, 156, 14, 24, 64, 2, 27, 598, 1152, 23, 4, 1375, 109, 7, 3140, 21, 12, 82, 2, 4, 311, 6, 2434, 2618, 83, 3140, 17, 6, 132, 7, 27, 273, 5, 58, 6, 132, 2, 17, 257, 31, 7, 178, 2, 9, 11, 4, 1341, 5, 2, 7, 27, 58, 5, 2987, 27, 96, 8, 2, 98, 45, 24, 38, 76, 143, 2, 2, 715, 5, 1729, 21, 11, 4, 1590, 4, 3393, 7, 4, 696, 1850, 15, 2108, 716, 4, 65, 7, 4, 2, 7, 4, 185, 2, 2, 281, 7, 273, 11, 926, 5, 2, 5, 7, 89, 4, 84, 2, 8, 443, 2, 715, 5, 2, 5, 2611, 68, 1684, 11, 14, 132, 937, 36, 2, 8, 28, 4300, 68, 1161, 5, 2, 10, 10, 202, 14, 6, 353, 305, 7, 4085, 23, 1469, 4499, 375, 58, 25, 650, 2, 443, 2, 2, 129, 1152, 8, 4, 696, 105, 25, 80, 13, 535, 30, 2, 2, 19, 6, 53, 3322, 4747, 7, 199, 3140, 5, 2108, 13, 40, 8, 1781, 15, 2, 2804, 143, 4, 696, 4395, 3393, 8, 2, 3140, 203, 28, 2, 8, 628, 2, 54, 29, 526, 2, 2, 11, 2, 2, 11, 63, 45, 4, 696, 4395, 3393, 8, 2, 15, 165, 376, 44, 2]
--- Label ---
1


Note IP: reviews are already preprocessed, details [here](https://keras.io/api/datasets/imdb/)

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [5]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

--- Review (with words) ---
['the', 'if', 'pleasure', 'for', 'but', 'and', 'called', 'and', 'television', 'record', 'parody', 'down', 'mouse', 'extremely', 'because', 'been', 'idea', 'before', 'as', 'his', 'see', 'and', 'be', 'complete', 'younger', 'are', 'of', 'chris', 'being', 'br', 'noble', 'not', 'that', 'other', 'and', 'of', 'night', 'is', '14', 'listening', 'first', 'noble', 'movie', 'is', 'say', 'br', 'be', 'put', 'to', 'my', 'is', 'say', 'and', 'movie', 'having', 'by', 'br', 'want', 'and', 'it', 'this', 'of', 'award', 'to', 'and', 'br', 'be', 'my', 'to', 'oddly', 'be', 'too', 'in', 'and', 'any', 'if', 'his', 'her', 'get', "i'm", 'and', 'and', 'jack', 'to', 'unusual', 'not', 'this', 'of', 'martin', 'of', 'rochester', 'br', 'of', 'television', 'record', 'for', 'parody', 'local', 'of', 'their', 'br', 'of', 'and', 'br', 'of', 'got', 'and', 'and', 'actor', 'br', 'put', 'this', 'footage', 'to', 'and', 'to', 'br', "don't", 'of', 'great', 'and', 'in', 'called', 'and', 'jack', 'to', 'an

In [7]:
max([len(review) for review in X_train])

2494

In [8]:
min([len(review) for review in X_train])

11

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

Range from 11 to 2,494


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [6]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [10]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [ ]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

# TODO: Train your model
model.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

In [ ]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(os.path.join(cache_dir, model_file))

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [ ]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!